In [ ]:

file_location = "/FileStore/tables/stations.csv"
file_type = "csv"

infer_schema = "false"
first_row_is_header = "false"
delimiter = ","

df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

df.write.format("delta") \
.save("/delta/bronze_stations")

In [ ]:

df = spark.read.format("delta") \
    .load("/delta/bronze_stations")

df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("silver_stations")

test = spark.sql("select * from silver_stations").show()

In [ ]:
%sql
--enabling property to update column name
ALTER TABLE silver_stations SET TBLPROPERTIES (
    'delta.minReaderVersion' = '2',
    'delta.minWriterVersion' = '5',
    'delta.columnMapping.mode' = 'name'
  )

In [ ]:
%sql
--renaming default column names to reflect more descriptive, column names
ALTER TABLE silver_stations 
RENAME COLUMN _c0 TO station_id;

ALTER TABLE silver_stations 
RENAME COLUMN _c1 TO station_name;

ALTER TABLE silver_stations 
RENAME COLUMN _c2 TO latitude;

ALTER TABLE silver_stations 
RENAME COLUMN _c3 TO longitude;

In [ ]:

tripsTbl = spark.table("silver_trips")
stnTbl = spark.table("silver_stations")
stnTbl2 = spark.table("silver_stations")

from pyspark.sql.functions import *
dim_stn_df = tripsTbl.join(stnTbl, tripsTbl.start_station_id == stnTbl.station_id,"inner") \
    .join(stnTbl2, tripsTbl.end_station_id == stnTbl2.station_id,"inner") \
    .select(stnTbl["station_id"].alias("station_key"), stnTbl["station_id"], stnTbl["station_name"], stnTbl["latitude"].alias("lat_station_location"), stnTbl["longitude"].alias("long_station_location"), tripsTbl["start_station_id"], tripsTbl["end_station_id"])
                
dim_stn_df.show()

dim_stn_df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("dim_stations")